We first import our previous work and the tf.keras backend.

In [1]:
from generate_data import *
from tensorflow import keras

Now generate the testing/training samples.

In [17]:
n_train, n_test = 10000, 2000

#generate 20000 training samples
x, y = create_data(true_sample_size = n_train + n_test)
x_tr, x_v, y_tr, y_v = x[:2*n_train], x[2*n_train:], y[:2*n_train], y[2*n_train:]

print(x[0],y[0])

<tf.RaggedTensor [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]> [0.]


In [19]:
print(len(y),len(y_tr),len(y_v))
print(n_train, n_test)

24000 20000 4000
10000 2000


Now define and compile the model using the adam optimiser and binary_crossentropy loss. (I experimented with dropout here to see how it impacts the results: I got a validation loss of about 1e-2 with dropout and about 1e-5 without dropout. Tweaking the dropout rate may help, but it seems dropout is counterproductive here)

In [24]:
#poss_chars = ['b', 't', 's', 'x', 'p', 'v', 'e']

model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None,8],dtype=tf.float32,ragged=True),
#    keras.layers.Embedding(input_dim=len(poss_chars),output_dim=6),
    keras.layers.GRU(32,return_sequences=True,dropout=0.1,
                     recurrent_dropout=0.1),
    keras.layers.GRU(32,dropout=0.1,recurrent_dropout=0.1),
    keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['binary_accuracy'])


In [ ]:
model.fit(x_tr,y_tr,epochs=10,validation_data=(x_v,y_v))

Epoch 1/10
625/625 [==============================] - 26s 42ms/step - loss: 0.6844 - binary_accuracy: 0.5464 - val_loss: 0.6130 - val_binary_accuracy: 0.8190
Epoch 2/10
625/625 [==============================] - 24s 38ms/step - loss: 0.6440 - binary_accuracy: 0.6237 - val_loss: 0.6010 - val_binary_accuracy: 0.5400
Epoch 3/10
625/625 [==============================] - 26s 41ms/step - loss: 0.5802 - binary_accuracy: 0.6878 - val_loss: 0.2087 - val_binary_accuracy: 0.9630
Epoch 4/10
625/625 [==============================] - 26s 41ms/step - loss: 0.2491 - binary_accuracy: 0.8995 - val_loss: 0.0257 - val_binary_accuracy: 1.0000
Epoch 5/10
625/625 [==============================] - 31s 49ms/step - loss: 0.1126 - binary_accuracy: 0.9704 - val_loss: 0.0132 - val_binary_accuracy: 1.0000
Epoch 6/10
625/625 [==============================] - 28s 45ms/step - loss: 0.0836 - binary_accuracy: 0.9804 - val_loss: 0.0115 - val_binary_accuracy: 1.0000
Epoch 7/10
625/625 [==============================] 

The model seems to almost perfectly learn the reber grammar (at least without dropout); we would expect this given the high complexity of the model and the simpleness of the grammar.

I acknowledge this neural net is highly overkill (especially given that I wrote a simple function to do the same task), however I have learned a lot about the tf tools available for preparing data and about some of the hyperparameters available when training RNNs.